In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [ ]:
# Veri setinin bulunduğu dizin
data_directory = '/content/drive/MyDrive/Pneumonia/X-Ray'

In [ ]:
# Veri artırma (data augmentation) işlemi
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Veri artırma ve normalizasyon
train_generator = train_datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation')

In [ ]:
# Yapay Sinir Ağı modelini oluşturma
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Modeli eğitme
history = model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)

In [ ]:
# Test verisi için sadece rescale (ölçeklendirme) işlemi yapma
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Test verisi üzerinde modelin performansını değerlendirme
test_generator = train_datagen.flow_from_directory(
        data_directory,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary',
        subset='validation',  # Test verisi için validation setini kullanabiliriz
        shuffle=False)

test_images = []
test_labels = []

for i in range(len(test_generator)):
    images, labels = test_generator[i]
    test_images.extend(images)
    test_labels.extend(labels)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

predictions = model.predict(test_images)
predicted_labels = (predictions > 0.5).astype('int').reshape(-1)


In [ ]:
# Performans metriklerini hesaplama
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)
conf_matrix = confusion_matrix(test_labels, predicted_labels)
class_report = classification_report(test_labels, predicted_labels)

In [ ]:

# Sonuçları gösterme
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")